In [2]:
from PyQt5.QtCore import QStandardPaths

config_path = QStandardPaths.writableLocation(QStandardPaths.AppDataLocation)
print(config_path)

C:/Users/mrkev/AppData/Roaming


In [3]:
from PyQt5.QtCore import QSettings

settings = QSettings("MySoft", "Star Runner")
settings.setValue("username", "user123")


In [9]:
settings.fileName()

'\\HKEY_CURRENT_USER\\Software\\MySoft\\Star Runner'

In [18]:
test = settings.value("username")
print(test)

None


In [13]:
settings.setValue("main/test", "hello")

In [19]:
settings.allKeys()

[]

In [1]:
import logging
print(logging.INFO)

20


In [2]:
from core.models import Scenario

In [7]:
for k, v in Scenario.__mapper__.relationships.items():
    print(v, type(v))

Scenario.bus_values <class 'sqlalchemy.orm.relationships._RelationshipDeclared'>
Scenario.branch_values <class 'sqlalchemy.orm.relationships._RelationshipDeclared'>
Scenario.machine_values <class 'sqlalchemy.orm.relationships._RelationshipDeclared'>
Scenario.two_winding_transformer_values <class 'sqlalchemy.orm.relationships._RelationshipDeclared'>
Scenario.case_links <class 'sqlalchemy.orm.relationships._RelationshipDeclared'>


In [ ]:
from pss_cli.core.database import db

: 

In [8]:
from typing import List, Optional, Sequence, Union
import src.pss_cli.core.models as models
from sqlmodel import SQLModel, Session, select, create_engine
from sqlalchemy import ColumnElement


class Database:
    def __init__(self, sqlite_filename: str):
        sqlite_url = f"sqlite:///{sqlite_filename}"
        self.engine = create_engine(sqlite_url)

    def get_all_table_names(self) -> List[str]:
        """Return a list of all table names"""
        return list(map(str, SQLModel.metadata.tables.keys()))

    def get_table_object(self, table_name: str):
        """Return a SQLModel of the table from the table name"""

        tables_dict = {
            str(table.__tablename__): table for table in SQLModel.__subclasses__()
        }
        return tables_dict.get(table_name)

    def create_db_and_tables(self):
        SQLModel.metadata.create_all(bind=self.engine)

    def select_table(
        self,
        table_name: str,
        where: Optional[ColumnElement[bool]] = None,
        session: Optional[Session] = None,
    ) -> Union[Sequence[SQLModel], Sequence[None], None]:
        """Return objects from the database table"""

        table_obj = self.get_table_object(table_name)
        statement = select(table_obj)
        if where is not None:
            statement = statement.where(where)

        if not session:
            with self.session() as session:
                results = session.exec(statement).all()
        else:
            results = session.exec(statement).all()

        if None in results:
            return None

        return results

    def session(self):
        """Return a session object"""
        return Session(self.engine)

    def add(self, obj: SQLModel, session: Session, commit: bool = True):
        """Add an object to the database"""

        session.add(obj)

        if commit:
            session.commit()
            session.refresh(obj)
        # print(f"Added {obj.__tablename__} object to the database:")
        # print_model(obj)
        return obj

    def delete(self, obj: SQLModel, session: Session, commit: bool = True):
        """Delete an object from the database"""

        session.delete(obj)

        if commit:
            session.commit()

    def commit(self, session: Session):
        """Commit the session"""
        session.commit()


db = Database(sqlite_filename="sqlite.db")


ModuleNotFoundError: No module named 'pss_cli'

In [6]:
db.create_db_and_tables()

In [7]:
db.get_all_table_names()

[]